<a href="https://www.kaggle.com/code/nizarkarkar/building-a-regression-mlp-using-the-sequential-api?scriptVersionId=100030931" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


**In many times we use the neural network models to classify images for example but what about using neural networks in machine learning problems like regression tasks.In this notebook we will see how to implement neural network  model to predict the SalePrcie of houses SO i m not going to fucus on data visulization the main objective is to show how to implement neural network model in a regression problem  **

In [2]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
%matplotlib inline 
import seaborn as sns 
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split 
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder

In [3]:
train=pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
train.drop('Id',axis=1, inplace=True)

In [4]:
test=pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')
test.drop('Id',axis=1, inplace=True)

In [5]:
train.shape,test.shape

((1460, 80), (1459, 79))

In [6]:
train.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [7]:
missing=train.isnull().sum().sort_values(ascending=False)
missing=missing.drop(missing[missing==0].index)
missing

PoolQC          1453
MiscFeature     1406
Alley           1369
Fence           1179
FireplaceQu      690
LotFrontage      259
GarageYrBlt       81
GarageCond        81
GarageType        81
GarageFinish      81
GarageQual        81
BsmtExposure      38
BsmtFinType2      38
BsmtCond          37
BsmtQual          37
BsmtFinType1      37
MasVnrArea         8
MasVnrType         8
Electrical         1
dtype: int64

In [8]:
def filling_missing_values(df):
    
    df['PoolQC']      =df['PoolQC'].fillna('no')
    df['MiscFeature'] =df['MiscFeature'].fillna('no')
    df['Alley']       =df['Alley'].fillna('no')
    df['Fence']       =df['Fence'].fillna('no')
    df['FireplaceQu'] =df['FireplaceQu'].fillna('no')
    df['GarageCond']  =df['GarageCond'].fillna('no')
    df['GarageQual']  =df['GarageQual'].fillna('no')
    df['GarageFinish']=df['GarageFinish'].fillna('no')
    df['BsmtExposure']=df['BsmtExposure'].fillna('no')
    df['BsmtCond']    =df['BsmtCond'].fillna('no')
    df['BsmtQual']    =df['BsmtQual'].fillna('no')
    df['BsmtFinType2']=df['BsmtFinType2'].fillna('no')
    df['BsmtFinType1']=df['BsmtFinType1'].fillna('no')
    df['Fence']       =df['Fence'].fillna('no')
    df['MasVnrType']  =df['MasVnrType'].fillna('no')
    df['GarageYrBlt'] =df['GarageYrBlt'].fillna(0)
    df['GarageType']  =df['GarageType'].fillna(0)
    df['GarageArea']  =df['GarageArea'].fillna(0)
    df['GarageCars']  =df['GarageCars'].fillna(0)
    df['BsmtFinSF1']  =df['BsmtFinSF1'].fillna(0)
    df['BsmtFinSF2']  =df['BsmtFinSF2'].fillna(0)
    df['MasVnrArea']  =df['MasVnrArea'].fillna(0)
    df['BsmtFullBath']=df['BsmtFullBath'].fillna(0)
    df['BsmtHalfBath']=df['BsmtHalfBath'].fillna(0)
    df['BsmtUnfSF']   =df['BsmtUnfSF'].fillna(0)
    df['TotalBsmtSF'] =df['TotalBsmtSF'].fillna(0)
    return df

In [9]:
train=filling_missing_values(train)
test=filling_missing_values(test)

In [10]:
train.drop('LotFrontage',axis=1, inplace=True)
train.drop('Electrical',axis=1, inplace=True)
test.drop('LotFrontage',axis=1, inplace=True)
test.drop('Electrical',axis=1, inplace=True)


In [11]:
cat_train=train.select_dtypes(include='object')
cat_test=test.select_dtypes(include='object')

In [12]:
from category_encoders import CountEncoder
enc = CountEncoder(normalize=True, cols=cat_train.columns)
enc2= CountEncoder(normalize=True, cols=cat_test.columns)
train = enc.fit_transform(train)
test = enc2.fit_transform(test)


In [13]:
training=train.drop('SalePrice',axis=1)
target_train=train['SalePrice']

In [14]:
X_train, X_test, y_train, y_test = train_test_split(training, target_train, test_size=0.33, random_state=0)

In [15]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)


# building a neural network 

Unlike classification, regression problems cannot be evaluated by accuracy. Here I will use Root Mean Squared Error as the evaluation metric.

In [16]:
import tensorflow as tf 
from tensorflow import keras 
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras import metrics

In [17]:
        model = Sequential()
        model.add(Dense(10, input_dim=X_train.shape[1], activation='relu'))
        model.add(Dense(30, activation='relu'))
        model.add(Dense(40, activation='relu'))
        model.add(Dense(1))
        model.compile(optimizer ='adam', loss = 'mean_squared_error', 
              metrics =[metrics.mae])

2022-07-04 20:56:33.507627: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                780       
_________________________________________________________________
dense_1 (Dense)              (None, 30)                330       
_________________________________________________________________
dense_2 (Dense)              (None, 40)                1240      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 41        
Total params: 2,391
Trainable params: 2,391
Non-trainable params: 0
_________________________________________________________________


In [19]:
history = model.fit(X_train, y_train, validation_data=(X_test,y_test), epochs=100, batch_size=32)

Epoch 1/100


2022-07-04 20:56:33.748384: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


31/31 [==============================] - 1s 10ms/step - loss: 39133483008.0000 - mean_absolute_error: 181409.6250 - val_loss: 38846713856.0000 - val_mean_absolute_error: 179926.4531
Epoch 2/100
31/31 [==============================] - 0s 3ms/step - loss: 39132020736.0000 - mean_absolute_error: 181406.2031 - val_loss: 38843994112.0000 - val_mean_absolute_error: 179920.3438
Epoch 3/100
31/31 [==============================] - 0s 3ms/step - loss: 39126077440.0000 - mean_absolute_error: 181393.4375 - val_loss: 38832713728.0000 - val_mean_absolute_error: 179897.1250
Epoch 4/100
31/31 [==============================] - 0s 3ms/step - loss: 39104188416.0000 - mean_absolute_error: 181349.7188 - val_loss: 38795022336.0000 - val_mean_absolute_error: 179822.8906
Epoch 5/100
31/31 [==============================] - 0s 4ms/step - loss: 39039086592.0000 - mean_absolute_error: 181223.8438 - val_loss: 38693826560.0000 - val_mean_absolute_error: 179627.7500
Epoch 6/100
31/31 [===========================

I also tried another DNN model and as we see this model gave me better result in mean squared error than the pervious models

In [20]:
model2= Sequential([
    tf.keras.layers.Input(shape = X_train.shape[1:]),
    Dense(300, activation='tanh'),
    Dense(300, activation='tanh'),
    Dense(300, activation='tanh'),
    Dense(300, activation='tanh'),
    Dense(300, activation='tanh'),
    Dense(300, activation='tanh'),
    Dense(1)
])

In [21]:
 model2.compile(optimizer ='adam', loss = 'mean_squared_error', 
              metrics =[metrics.mae])

In [22]:
history2 = model.fit(X_train, y_train, validation_data=(X_test,y_test), epochs=100, batch_size=32)

Epoch 1/100
31/31 [==============================] - 0s 5ms/step - loss: 1135740544.0000 - mean_absolute_error: 24665.9570 - val_loss: 2138529152.0000 - val_mean_absolute_error: 29530.4062
Epoch 2/100
31/31 [==============================] - 0s 4ms/step - loss: 1125796864.0000 - mean_absolute_error: 24555.8398 - val_loss: 2129425664.0000 - val_mean_absolute_error: 29404.7285
Epoch 3/100
31/31 [==============================] - 0s 4ms/step - loss: 1115743104.0000 - mean_absolute_error: 24400.5352 - val_loss: 2121935232.0000 - val_mean_absolute_error: 29271.9844
Epoch 4/100
31/31 [==============================] - 0s 4ms/step - loss: 1107647744.0000 - mean_absolute_error: 24326.4980 - val_loss: 2115253120.0000 - val_mean_absolute_error: 29165.4102
Epoch 5/100
31/31 [==============================] - 0s 4ms/step - loss: 1097519488.0000 - mean_absolute_error: 24168.8301 - val_loss: 2107783296.0000 - val_mean_absolute_error: 29055.3574
Epoch 6/100
31/31 [==============================] - 0s

In [23]:
input = tf.keras.layers.Input(shape = X_train.shape[1:])
hidden1 = tf.keras.layers.Dense(300, activation='relu')(input)
hidden2 = tf.keras.layers.Dense(300, activation='relu')(hidden1)
hidden3 = tf.keras.layers.Dense(300, activation='relu')(hidden2)
hidden4 = keras.layers.Concatenate()([input, hidden3])
hidden5 = tf.keras.layers.Dense(300, activation='relu')(hidden4)
concat = keras.layers.Concatenate()([input, hidden5])
output = keras.layers.Dense(1)(concat)
model3 = keras.models.Model(inputs=[input], outputs=[output])

In [24]:
 model2.compile(optimizer ='adam', loss = 'mean_squared_error', 
              metrics =[metrics.mae])

In [25]:
history3 = model.fit(X_train, y_train, validation_data=(X_test,y_test), epochs=100, batch_size=32)

Epoch 1/100
31/31 [==============================] - 0s 5ms/step - loss: 628122432.0000 - mean_absolute_error: 17760.7539 - val_loss: 1794826624.0000 - val_mean_absolute_error: 23820.7871
Epoch 2/100
31/31 [==============================] - 0s 4ms/step - loss: 626882112.0000 - mean_absolute_error: 17759.1191 - val_loss: 1794870016.0000 - val_mean_absolute_error: 23807.0977
Epoch 3/100
31/31 [==============================] - 0s 4ms/step - loss: 621599808.0000 - mean_absolute_error: 17639.9434 - val_loss: 1789288064.0000 - val_mean_absolute_error: 23717.6895
Epoch 4/100
31/31 [==============================] - 0s 4ms/step - loss: 620205952.0000 - mean_absolute_error: 17594.5938 - val_loss: 1790694144.0000 - val_mean_absolute_error: 23731.2480
Epoch 5/100
31/31 [==============================] - 0s 4ms/step - loss: 616630016.0000 - mean_absolute_error: 17542.8691 - val_loss: 1790379520.0000 - val_mean_absolute_error: 23723.3750
Epoch 6/100
31/31 [==============================] - 0s 4ms/

Well this model looks like it doesn't add much to the reduction of the mean squared error but it performs better then model number 2